# Fortsetzung BeautifulSoup

Letzte Woche haben wir alle Zitate von der ersten Seite der Website [https://quotes.toscrape.com](https://quotes.toscrape.com) extrahiert.
Heute werden wir den Code in zwei Aspekten ergänzen:

1. Zitate von der ersten Seite mit Metadaten extrahieren
2. Zitate von allen Seiten extrahieren, mit und ohne Metadaten
3. Daten in Dateien schreiben: Beispiel pandas DataFrame in Excel-Tabelle

Für die Ausführung des Codes brauchen wir zusätzlich den Paketen requests und bs4 außerdem noch das Paket pandas, und eine Funktion aus dem Paket urllib3. Zusätzlich könnt ihr das Paket memory_profiler installieren, das erlaubt, zu messen, wieviel Speicher zum Ausführen einer Codezelle benötigt wird. Daneben installieren wir ein Paket openpyxl, das zum Schreiben von pandas-DataFrames in Excel-Dateien verwendet wird. Das Paket müssen wir aber nicht laden, weil es automatisch geladen wird, wenn wir später versuchen, einen Pandas DataFrame in eine Excel-Datei zu schreiben.

In [ ]:
#import sys
#!conda install --yes --prefix {sys.prefix} memory_profiler
#!conda install --yes --prefix {sys.prefix} urllib3
#!conda install --yes --prefix {sys.prefix} openpyxl

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlsplit # brauchen wir nur für die Funktion scrape_all_urls()
# %load_ext memory_profiler

## Recap: Zitate von der ersten Seite extrahieren, ohne Metadaten

In [ ]:
URL = "https://quotes.toscrape.com/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

zitate = soup.find_all('span', class_="text")

for zitat in zitate:
    print(zitat.get_text())

## Zitate von der ersten Seite extrahieren, mit Metadaten

In der letzten Stunde haben wir die BeautifulSoup-Methoden .find() und .find_all() verwendet, um HTML-Elemente in einem  BeautifulSoup-Objekt zu finden. Genauso könnten wir auch vorgehen, um neben den Zitaten auch einige Metadaten zu extrahieren, also zum Beispiel auch den Namen der Person, von der das Zitat stammt, und die Tags, mit denen das Zitat versehen wurde.

In [ ]:
quotes = soup.find_all('div', class_='quote')
quotes_dict = {"Text":[], "Author":[], "Tags":[]}

for quote in quotes:
    quote_text = quote.find('span', class_='text').get_text()
    quote_author = quote.find('small', class_='author').get_text()
    quote_tags = quote.find_all('a', class_='tag')
    tags_text = []
    for tag in quote_tags:
        tags_text.append(tag.get_text())
    quotes_dict["Text"].append(quote_text)
    quotes_dict["Author"].append(quote_author)
    quotes_dict["Tags"].append(tags_text)


In [ ]:
# dictionary ist nicht besonders übersichtlich
quotes_dict

In [ ]:
# Dataframe ist übersichtlicher
quotes_df = pd.DataFrame.from_dict(quotes_dict)
quotes_df

Anstelle der Methode .get_text() kann auch das Attribut .text abgerufen werden: Beide geben den Textinhalt des Elements zurück.

Aber was passiert, wenn ein Element nicht gefunden werden kann, beispielsweise, weil für ein Zitat keine Tags angegeben wurden, oder wenn die Angabe der Autor:in fehlt? In diesem Fall würden die Methoden .find() bzw. .find_all() den Wert None zurückgeben. Die Methode .get_text() (oder das Attribut .text) würde dann auf ein Objekt vom Typ NoneType angewandt werden. Aber NoneType-Objekte haben keine Methode .get_text() und auch kein Attribut .text! Der Code würde also eine Fehlermeldung produzieren und die Ausführung abbrechen. Um das zu verhindern, könnten wir zunächst überprüfen, ob tatsächlich ein Element gefunden wurde. Nur, wenn ein Element gefunden wurde, wird der Text extrahiert, im folgenden Beispiel mithilfe des Attributs .text statt der Methode .get_text():

In [ ]:
quotes = soup.find_all('div', class_='quote')
quotes_dict = {"Text":[], "Author":[], "Tags":[]}

for quote in quotes:
    quote_text = quote.find('span', class_='text')
    quote_author = quote.find('small', class_='author')
    quote_tags = quote.find_all('a', class_='tag')
    tags_text = []
    for tag in quote_tags:
        tags_text.append(tag.text)
    if quote_text is not None:
        quotes_dict["Text"].append(quote_text.text)
    if quote_author is not None:
        quotes_dict["Author"].append(quote_author.text)
    if len(tags_text) != 0:
        quotes_dict["Tags"].append(tags_text)


Die Suche mit find() und find_all() produziert bei komplexeren Abfragen aber unübersichtlichen Code, weil wir als Argument zusätzlich die CSS-Klasse des gesuchten Elements angeben müssen. Eine einfachere Möglichkeit, direkt nach der CSS-Klasse selbst zu suchen, sind die Methoden .select_one() und .select().

Zum Nachlesen: [https://beautiful-soup-4.readthedocs.io/en/latest/index.html?highlight=select#css-selectors](https://beautiful-soup-4.readthedocs.io/en/latest/index.html?highlight=select#css-selectors)

In [ ]:
quotes = soup.select('div.quote')
quotes_dict = {"Text":[], "Author":[], "Tags":[]}

for quote in quotes:
    quote_text = quote.select_one('span.text') # oder einfach '.text'
    quote_author = quote.select_one('small.author')
    quote_tags = quote.select('a.tag')
    tags_text = []
    for tag in quote_tags:
        tags_text.append(tag.text)
    if quote_text is not None:
        quotes_dict["Text"].append(quote_text.text)
    if quote_author is not None:
        quotes_dict["Author"].append(quote_author.text)
    if len(tags_text) != 0:
        quotes_dict["Tags"].append(tags_text)

Die Suche nach CSS-Selektoren mithilfe der .select_one() und .select() Methoden hat außerdem einen weiteren Vorteil: Sie erlauben, direkt nach Kind- oder Geschwisterelementen eines Elements zu suchen. Bei der Verwendung von .find() und .find_all() hatten wir eine for-Schleife verwendet, um die Suche auf Kindelemente der div-Elemente mit der Klasse 'quote' einzuschränken. In manchen Fällen kann die Verwendung von .select() anstelle von .find_all() eine solche for-Schleife ersetzen. In unserem Beispiel würde zur Suche nach Zitattexten und Autor:innennamen beispielsweise keine for-Schleife erfordern:

In [ ]:
# Zitattexte: 'div.quote > span.text' findet direkte Kindelemente des div-Elements mit der Klasse 'quote'
soup.select('div.quote > span.text')

In [ ]:
# Autor:innen: 'div.quote small.author' findet alle small-Elemente mit der Klasse 'author' innerhalb de des div-Elements mit der Klasse 'quote', auch "Enkelkinder"
soup.select('div.quote small.author')

## Zitate von allen Seiten der Website extrahieren, ohne Metadaten

### Lösung mit for-Schleife: für exakt 10 Unterseiten

In [ ]:
%%time
# %%memit
# Lösungsidee von GitHub-Nutzer:in Bhavya Bindela: https://bhavyasree.github.io/PythonClass/Notebooks/18.scrape-quotes/ . Angepasst für die Extraktion von Zitaten statt Autor:innen
# for Schleife mit Set

base_url = 'http://quotes.toscrape.com/page/'

quotes = set()

for i in range(1,11):
    scrape_url = base_url + str(i)
    page = requests.get(scrape_url)
    soup = BeautifulSoup(page.content, "html.parser")

    for quote in soup.select('.quote > .text'):
        quotes.add(quote.text) # type(quote) ist bs4.element.Tag: hat Attribut text; add() ist eine set-Methode

In [ ]:
quotes # sets haben keine Ordnung: das ist unpraktisch

In [ ]:
# haben wir alle zitate extrahiert?
len(quotes)

Diese Lösung nutzt ein python-Set, um die extrahierten Elemente zu speichern. Das ist allerdings etwas unpraktisch, da Sets ungeordnet sind und die Zitate so nicht in chronologischer Reihenfolge gespeichert werden. Es empfiehlt sich deswegen, stattdessen eine Liste zu verwenden:

In [ ]:
%%time
# %%memit

base_url = 'http://quotes.toscrape.com/page/'

quotes = []

for i in range (1,11):
    scrape_url = base_url + str(i)
    page = requests.get(scrape_url)
    soup = BeautifulSoup(page.content, "html.parser")

    for quote in soup.select('.quote > .text'):
        quotes.append(quote.text)


In [ ]:
quotes

Die Ausgabe von %%time und %%memit zeigen, dass die Nutzung des Sets keinen (Effizienz-) Vorteil gegenüber Listen hat: Wir können also genausogut eine Liste verwenden.

### Lösung mit while-Schleife: unbekannte Anzahl von Unterseiten

In [ ]:
%%time
# %%memit
# while-Schleife mit Set
# Lösung wieder von GitHub-Nutzer:in Bhavya Bindela: https://bhavyasree.github.io/PythonClass/Notebooks/18.scrape-quotes/ . Angepasst für die Extraktion von Zitaten statt Autor:innen

scrape_url = base_url + str(999999)
page = requests.get(scrape_url)
soup = BeautifulSoup(page.content, "html.parser")
soup

page_no = 1
quotes = set()
base_url = 'http://quotes.toscrape.com/page/'

while True:
    scrape_url = base_url + str(page_no)
    page = requests.get(scrape_url)

    # Das funktioniert nur für die Seite quotes.toscrape.com
    # Für andere Seiten könnte hier die Bedingung if page.status_code != 200
    # getestet werden
    if 'No quotes found!' in page.text:
        break

    soup = BeautifulSoup(page.content, "html.parser")

    for quote in soup.select('.quote > .text'):
        quotes.add(quote.text)

    page_no +=1


In [ ]:
quotes

Die Zeile, die auf den ersten Blick verwundert, ist wahrscheinlich die if-Anweisung: if 'No quotes found!' in page.text: break.
Was hat es damit auf sich?
Die Macher:innen der Seite quotes.toscrape haben sich überlegt, dass auch Seiten, auf denen keine Zitate mehr publiziert sind, existieren sollen, sodass eine HTTP-Anfrage für diese Seiten einen Erfolgscode 200 zurückgeben. Wenn die Seiten nicht existieren würden, könnte einfach die while-Schleife in Abhängigkeit von dem Statuscode abgebrochen werden.

Das können wir im Vergleich mit einer anderen Seite illustrieren:

In [ ]:
# warum if 'No quotes found!' ...?
# Es gibt eine seite 99999: Das ist nur ausnahmsweise auf der Seite quotes.toscrape so.
page = requests.get('http://quotes.toscrape.com/page/9999')
page.status_code

In [ ]:
# Auf dieser Seite steht ein einziger Satz
# Durchsucht den String nach dem Satz: Welcher ist es?
page.text

In [ ]:
# Anders wäre es z.B. hier:
page = requests.get('https://www.projekt-gutenberg.org/balzac/kurtisa2/chap001.html')
page.status_code # 200
# Es gibt keine Seite 99999
page = requests.get('https://www.projekt-gutenberg.org/balzac/kurtisa2/chap99999.html')
page.status_code # 404

Auch die while-Schleife können wir wieder zum Erstellen einer Liste anstelle eines Sets verwenden:

In [ ]:
%%time
# %%memit
# while-Schleife mit Liste

scrape_url = base_url + str(999999)
page = requests.get(scrape_url)
soup = BeautifulSoup(page.content, "html.parser")
soup

page_no = 1
quotes = []
base_url = 'http://quotes.toscrape.com/page/'

while True:
    scrape_url = base_url + str(page_no)
    page = requests.get(scrape_url)

    if 'No quotes found!' in page.text:
        break

    soup = BeautifulSoup(page.content, "html.parser")

    for quote in soup.select('.quote > .text'):
        quotes.append(quote.text)

    page_no +=1

In [ ]:
quotes

## Zitate von allen Seiten extrahieren, mit Metadaten

### Lösung mit for-Schleife

In [ ]:
%%time
# %%memit

base_url = 'http://quotes.toscrape.com/page/'

quotes_dict = {"Zitat":[], "Quelle":[], "Tags":[], "URL":[]}

for i in range (1,11):
    scrape_url = base_url + str(i)
    page = requests.get(scrape_url)
    soup = BeautifulSoup(page.content, "html.parser")

    quotes_divs = soup.select('.quote')
    for div in quotes_divs:
        quotes_dict["Zitat"].append(div.select_one('.text').get_text()) # können hier nicht nach css-Klasse suchen, weil es mehrere Elemente mit dem Klassennamen text gibt
        quotes_dict["Quelle"].append(div.select_one('.author').get_text()) #oder .text
        temp = []
        tags = div.select('.tags > a') # findet Kind-Elemente von dem Element mit der Klasse tags:  tags aus der Top Ten Tags-Liste werden nicht gefunden, weil wir bereits in der quote div sind
        for tag in tags:
            temp.append(tag.get_text())
        quotes_dict["Tags"].append(temp)
        quotes_dict["URL"].append(scrape_url)

quotes_df = pd.DataFrame.from_dict(quotes_dict)

In [ ]:
quotes_df

### Lösung mit while-Schleife

In [ ]:
%%time
# %%memit

scrape_url = base_url + str(999999)
page = requests.get(scrape_url)
soup = BeautifulSoup(page.content, "html.parser")
soup

page_no = 1
quotes_dict = {"Zitat":[], "Quelle":[], "Tags":[], "URL":[]}
base_url = 'http://quotes.toscrape.com/page/'

while True:
    scrape_url = base_url + str(page_no)
    page = requests.get(scrape_url)

    if 'No quotes found!' in page.text:
        break

    soup = BeautifulSoup(page.content, "html.parser")

    quote_divs = soup.select('.quote')
    for div in quote_divs:
        quotes_dict["Zitat"].append(div.select_one('.text').get_text()) # können hier nicht nach css-Klasse suchen, weil es mehrere Elemente mit dem Klassennamen text gibt
        quotes_dict["Quelle"].append(div.select_one('.author').get_text())
        temp = []
        tags = div.select('.tags > a')
        for tag in tags:
            temp.append(tag.get_text())
        quotes_dict["Tags"].append(temp)
        quotes_dict["URL"].append(scrape_url)

    page_no +=1

quotes_df = pd.DataFrame.from_dict(quotes_dict)

In [ ]:
quotes_df

### Lösung mit Funktionen

Bei der Lösung mit der for-Schleife mussten wir die genaue Anzahl von Unterseiten kennen. Bei der Lösung mit while-Schleife mussten wir nicht die Anzahl der Unterseiten kennen, alle Unterseiten mussten allerdings eine ähnliche URL haben, sodass wir in jedem Schleifendurchlauf einfach die Seitenzahl erhöhen konnten, um die URL für die nächste Seite zu generieren.

Aber was, wenn alle Unterseiten verschiedene URLs haben? Für diesen Fall können wir auf die Funktion `urlsplit()` aus dem Paket urllib3 zurückgreifen. Die Funktion teilt eine URL in ihre Bestandteile auf und gibt ein Tupel-Objekt zurück. Die Bestandteile der URL können als Attribute des Tupel-Objekts abgerufen werden. So können Bestandteile einer URL flexibel ausgetauscht werden.

Diese Funktion kann auch in Zusammenhang mit einer while-Schleife zum Scrapen von Websites mit Unterseiten, deren Pfade variieren, verwendet werden. Die Lösung mit Funktionsdefinitionen ist nur eine mögliche Lösung:

In [ ]:
def get_soup(url):
    """
    Argumente: `url` (String): Die URL der Webseite.
    Rückgabewert: `soup` (BeautifulSoup-Objekt): Das BeautifulSoup-Objekt, das den analysierten HTML-Inhalt der Webseite repräsentiert.

    Diese Funktion ruft den HTML-Inhalt der Webseite unter der angegebenen URL mit der requests.get()-Methode ab. Anschließend wird ein BeautifulSoup-Objekt erstellt, indem der HTML-Inhalt mit dem Parser "html.parser" analysiert wird. Das resultierende BeautifulSoup-Objekt wird zurückgegeben.
    """
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    return soup

In [1]:
def extract_quotes(soup, url, quotes_dict):
    """
    Argumente:
    `soup` (BeautifulSoup-Objekt): Das BeautifulSoup-Objekt, das den analysierten HTML-Inhalt einer Webseite repräsentiert.
    `url` (String): Die URL der Webseite, aus der die Zitate extrahiert werden.
    `quotes_dict` (Dictionary): Ein Wörterbuch, das die Zitate enthält.
    Rückgabewert:
    `quotes_dict` (Dictionary): Das aktualisierte Wörterbuch, das die extrahierten Zitate enthält.

    Die Funktion extrahiert Zitate, Autor:innen, Tags und URLs aus einem BeautifulSoup-Objekt. Sie wählt dazu bestimmte HTML-Elemente mit den entsprechenden Klassen aus und fügt die extrahierten Informationen in das quotes_dict-Wörterbuch ein, das anschließend zurückgegeben wird.
    """
    quotes_divs = soup.select('.quote')
    for div in quotes_divs:
        quotes_dict["Zitat"].append(div.select_one('.text').get_text())
        quotes_dict["Quelle"].append(div.select_one('.author').get_text())
        temp = []
        tags = div.select('.tags > a')
        for tag in tags:
            temp.append(tag.get_text())
        quotes_dict["Tags"].append(temp)
        quotes_dict["URL"].append(url)
    return quotes_dict

In [ ]:
def scrape_all_quotes(url, quotes_dict = None):
    """
    Argumente:
    `url` (String): Die URL der Webseite, von der die Zitate gesammelt werden sollen.
    `quotes_dict` (Dictionary, optional): Ein Wörterbuch, das die Zitate enthält. Wenn nicht angegeben, wird ein neues Wörterbuch erstellt.
    Rückgabewert:
    `quotes_df` (DataFrame): Ein Pandas DataFrame, der die gesammelten Zitate enthält.

    Die Funktion sammelt Zitate von der angegebenen Webseite und allen Unterseiten. Wenn kein quotes_dict-Wörterbuch bereitgestellt wird, wird ein neues Wörterbuch mit leeren Listen erstellt. Das BeautifulSoup-Objekt wird über get_soup() abgerufen und die extract_quotes()-Funktion extrahiert die Zitate und aktualisiert das Wörterbuch. Wenn keine nächste Seite vorhanden ist (bestimmt durch das Fehlen des ".next"-Elements), wird ein Pandas DataFrame (quotes_df) aus dem quotes_dict erstellt und zurückgegeben. Andernfalls wird die URL der nächsten Seite abgerufen und scrape_all_quotes() rekursiv mit der nächsten Seiten-URL und dem aktuellen quotes_dict aufgerufen.
    """
    if quotes_dict is None:
        quotes_dict = {"Zitat":[], "Quelle":[], "Tags":[], "URL":[]}
    soup = get_soup(url)
    quotes_dict = extract_quotes(soup, url, quotes_dict)
    if not soup.select_one('.next'):
        quotes_df = pd.DataFrame.from_dict(quotes_dict)
        return quotes_df
    else:
        nextpage = soup.select_one('.next > a').get('href')
        url_parts = urlsplit(url)
        baseurl = url_parts.scheme + '://' + url_parts.netloc
        nextpage_url = baseurl + nextpage
        return scrape_all_quotes(nextpage_url, quotes_dict)

In [ ]:
%%time
# %%memit
df = scrape_all_quotes("https://quotes.toscrape.com")

In [ ]:
df

Die Lösung mit Funktionsdefinitionen ist zwar flexibler, weil wir die Pfade der Unterseiten nicht kennen müssen. Diese Lösung erfodert aber auch etwas mehr Laufzeit als die Lösung mit Schleifen. Die Funktion extract_quotes() können wir aber zumindest noch etwas effizienter machen, indem wir die Schleife `for tag in tags ...` durch ein etwas effizienteres Konstrukt ersetzten, das sich "List Comprehension" nennt. Was das genau ist, lernen wir nächste Stunde. So sieht die Funktion extract_quotes() mit list comprehension aus:

In [ ]:
# Funktion extract_quotes() mit list comprehension
def extract_quotes(soup, url, quotes_dict):
    quotes_divs = soup.select('.quote')
    for div in quotes_divs:
        quotes_dict["Zitat"].append(div.select_one('.text').get_text())
        quotes_dict["Quelle"].append(div.select_one('.author').get_text())
        [tag.get_text() for tag in div.select('.tags > a')]
        quotes_dict["Tags"].append(tags)
        quotes_dict["URL"].append(url)
    return quotes_dict

```{note}
In einer vorigen Version dieser Seite wurde das leere `quotes_dict` direkt in der Definition der Funktion `scrape_all_quotes()` als Defaultargument definiert. Dabei hatte ich allerdings eine Verhaltensweise von Python-Funktionen vergessen: Wenn Defaultargumente einen veränderbaren Datentyp haben, werden sie in Python bei einem wiederholten Funktionsaufruf "mitgenommen" und nicht durch den Default-Wert ersetzt. Das heißt, dass bei jedem erneuten Funktionsaufruf der `quotes_df` DataFrame wächst, weil das `quotes_dict` nicht nur die Elemente des aktuellen Funktionsaufrufs, sondern auch die Elemente aller vorhergegangener Funktionsaufurfe enthält. Statt direkt das Dictionary als Defaultargument festzulegen, sollte deswegen lieber zunächst None als Defaultwert festgelegt werden. Das Dictionary kann dann mithilfe einer bedingten Anweisung im Funktionskörper erstellt werden, nämlich genau dann, wenn das `quotes_dict` beim Funktionsaufruf None ist. Das ist nur beim ersten Funktionsaufruf der Fall.

Mehr zum Umgang mit Defaultargumenten: [https://docs.python-guide.org/writing/gotchas/#mutable-default-arguments](https://docs.python-guide.org/writing/gotchas/#mutable-default-arguments)
```

## Pandas-DataFrame in Exceldatei schreiben

Um einen Pandas DataFrame zu speichern, gibt es verschiedene Methoden. Die Methode .to_excel() erlaubt zum Beispiel, einen DataFrame in einer Exceltabelle zu speichern, also in einer Datei mit der Dateiendung .xlsx. Für "speichern" sagt man in diesem Kontext auch "schreiben": Daten werden in eine Datei geschrieben.

Die Methode .to_excel() greift unter der Motorhaube auf ein Paket mit dem Namen openpyxl zurück. Diese Paket mussten wir deswegen am Anfang installieren.

Dokumentation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html

In [ ]:
# quotes_df.to_excel("quotes_df.xlsx", index=False) # default-encoding ist UTF-8

## \%%time und \%%memit: Was ist das?

Wir haben beim Ausführen der Codezellen in diesem Jupyter Notebook jeweils zwei Zeilen am Anfang hinzugefügt:

- \%%time berechnet die Laufzeit einer Jupyter Notebook Codezelle.
- \%%memit berechnet, wie viel Speicher für die Ausführung der Codezelle benötigt wird.

Damit können wir vergleichen, welche Lösung am effizientesten ist. Mehr Informationen findet ihr wie immer in den Dokumentationsseiten:

- https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-time
- https://ipython-books.github.io/44-profiling-the-memory-usage-of-your-code-with-memory_profiler/